In [116]:
import numpy as np
import pandas as pd

import datetime

import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import  OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import r2_score
from sklearn.model_selection import cross_val_score, GridSearchCV

import warnings
warnings.filterwarnings('ignore')

In [29]:
df = pd.read_csv('Walmart_Store_sales.csv')
df.head()

,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
0,6.0,18-02-2011,1572117.54,NaN,59.61,3.045,214.777523,6.858
1,13.0,25-03-2011,1807545.43,0.0,42.38,3.435,128.616064,7.470
2,17.0,27-07-2012,NaN,0.0,NaN,NaN,130.719581,5.936
3,11.0,NaN,1244390.03,0.0,84.57,NaN,214.556497,7.346
4,6.0,28-05-2010,1644470.66,0.0,78.89,2.759,212.412888,7.092


In [30]:
df.shape

(150, 8)

In [31]:
display(df.info())

display(df.describe(include='all'))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Store         150 non-null    float64
 1   Date          132 non-null    object 
 2   Weekly_Sales  136 non-null    float64
 3   Holiday_Flag  138 non-null    float64
 4   Temperature   132 non-null    float64
 5   Fuel_Price    136 non-null    float64
 6   CPI           138 non-null    float64
 7   Unemployment  135 non-null    float64
dtypes: float64(7), object(1)
memory usage: 9.5+ KB


None

,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
count,150.000000,132,1.360000e+02,138.000000,132.000000,136.000000,138.000000,135.000000
unique,NaN,85,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,19-10-2012,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,4,NaN,NaN,NaN,NaN,NaN,NaN
mean,9.866667,NaN,1.249536e+06,0.079710,61.398106,3.320853,179.898509,7.598430
std,6.231191,NaN,6.474630e+05,0.271831,18.378901,0.478149,40.274956,1.577173
min,1.000000,NaN,2.689290e+05,0.000000,18.790000,2.514000,126.111903,5.143000
25%,4.000000,NaN,6.050757e+05,0.000000,45.587500,2.852250,131.970831,6.597500
50%,9.000000,NaN,1.261424e+06,0.000000,62.985000,3.451000,197.908893,7.470000
75%,15.750000,NaN,1.806386e+06,0.000000,76.345000,3.706250,214.934616,8.150000


In [32]:
df.isnull().sum()

Store            0
Date            18
Weekly_Sales    14
Holiday_Flag    12
Temperature     18
Fuel_Price      14
CPI             12
Unemployment    15
dtype: int64

# Part 1 : EDA

## Drop missing values target

In [33]:
# Suppression des lignes avec Weekly_Sales (target) à null
# 14 lignes
df.dropna(subset=['Weekly_Sales'], inplace=True)
df.isnull().sum()

Store            0
Date            18
Weekly_Sales     0
Holiday_Flag    11
Temperature     15
Fuel_Price      12
CPI             11
Unemployment    14
dtype: int64

## Create usable features from the Date column

In [34]:
df['Year'] = pd.to_datetime(df['Date']).dt.year
df['Month'] = pd.to_datetime(df['Date']).dt.month
df['Day'] = pd.to_datetime(df['Date']).dt.day
df['DayOfWeek'] = pd.to_datetime(df['Date']).dt.day_name()

df = df.drop('Date',axis=1)

df.head(5)

,Store,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment,Year,Month,Day,DayOfWeek
0,6.0,1572117.54,NaN,59.61,3.045,214.777523,6.858,2011.0,2.0,18.0,Friday
1,13.0,1807545.43,0.0,42.38,3.435,128.616064,7.470,2011.0,3.0,25.0,Friday
3,11.0,1244390.03,0.0,84.57,NaN,214.556497,7.346,NaN,NaN,NaN,NaN
4,6.0,1644470.66,0.0,78.89,2.759,212.412888,7.092,2010.0,5.0,28.0,Friday
5,4.0,1857533.70,0.0,NaN,2.756,126.160226,7.896,2010.0,5.0,28.0,Friday


In [35]:
df.describe(include='all')

,Store,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment,Year,Month,Day,DayOfWeek
count,136.000000,1.360000e+02,125.000000,121.000000,124.000000,125.000000,122.000000,118.000000,118.000000,118.000000,118
unique,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
top,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Friday
freq,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,118
mean,10.014706,1.249536e+06,0.072000,60.853967,3.316992,178.091144,7.665582,2010.822034,6.338983,16.440678,NaN
std,6.124614,6.474630e+05,0.259528,18.514432,0.479540,40.243105,1.619428,0.812628,3.173664,8.209378,NaN
min,1.000000,2.689290e+05,0.000000,18.790000,2.514000,126.111903,5.143000,2010.000000,1.000000,1.000000,NaN
25%,4.000000,6.050757e+05,0.000000,45.220000,2.838500,131.637000,6.690000,2010.000000,4.000000,10.000000,NaN
50%,10.000000,1.261424e+06,0.000000,62.250000,3.451000,196.919506,7.477000,2011.000000,6.000000,16.500000,NaN
75%,15.250000,1.806386e+06,0.000000,75.950000,3.724000,214.878556,8.150000,2011.750000,9.000000,24.000000,NaN


In [60]:
# Suppression de la colonne, contient que des vendredi
df = df.drop('DayOfWeek',axis=1)

## Drop lines containing invalid values or outliers

In [37]:
cols = ["Temperature","Fuel_Price","CPI","Unemployment"]

for col in cols:
    fig = px.box(df, y = col)
    fig.show()

In [38]:
for col in df[cols]:
    upper_limit = df[col].mean() + df[col].std() * 3 
    lower_limit = df[col].mean() - df[col].std() * 3
    df = df[(df[col] < upper_limit) & (df[col] > lower_limit)]

df.describe(include="all")

,Store,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment,Year,Month,Day,DayOfWeek
count,90.000000,9.000000e+01,80.000000,90.00000,90.000000,90.000000,90.000000,80.000000,80.000000,80.000000,80
unique,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
top,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Friday
freq,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,80
mean,9.900000,1.233865e+06,0.075000,61.06100,3.318444,179.524905,7.389733,2010.887500,6.362500,16.125000,NaN
std,6.204475,6.647250e+05,0.265053,17.74604,0.484399,39.554303,0.982729,0.826672,3.028321,8.521566,NaN
min,1.000000,2.689290e+05,0.000000,18.79000,2.548000,126.128355,5.143000,2010.000000,1.000000,1.000000,NaN
25%,4.000000,5.617240e+05,0.000000,45.34250,2.814750,132.602339,6.642250,2010.000000,4.000000,10.000000,NaN
50%,9.000000,1.260826e+06,0.000000,61.45000,3.468000,197.166416,7.419000,2011.000000,6.000000,16.500000,NaN
75%,15.750000,1.807159e+06,0.000000,75.79250,3.737750,214.855374,8.099000,2012.000000,8.250000,23.250000,NaN


In [61]:
display(df.shape)

display(df.isnull().sum())

(90, 10)

Store            0
Weekly_Sales     0
Holiday_Flag    10
Temperature      0
Fuel_Price       0
CPI              0
Unemployment     0
Year            10
Month           10
Day             10
dtype: int64

150 => 90

In [62]:
df["Store"] = df["Store"].apply(lambda x: int(x) if pd.notna(x) else x)
df["Holiday_Flag"] = df["Holiday_Flag"].apply(lambda x: int(x) if pd.notna(x) else x)

df

,Store,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment,Year,Month,Day
0,6,1572117.54,NaN,59.61,3.045,214.777523,6.858,2011.0,2.0,18.0
1,13,1807545.43,0.0,42.38,3.435,128.616064,7.470,2011.0,3.0,25.0
4,6,1644470.66,0.0,78.89,2.759,212.412888,7.092,2010.0,5.0,28.0
6,15,695396.19,0.0,69.80,4.069,134.855161,7.658,2011.0,6.0,3.0
7,20,2203523.20,0.0,39.93,3.617,213.023622,6.961,2012.0,2.0,3.0
...,...,...,...,...,...,...,...,...,...,...
139,7,532739.77,0.0,50.60,3.804,197.588605,8.090,2012.0,5.0,25.0
143,3,396968.80,0.0,78.53,2.705,214.495838,7.343,2010.0,6.0,4.0
144,3,424513.08,0.0,73.44,3.594,226.968844,6.034,2012.0,10.0,19.0
145,14,2248645.59,0.0,72.62,2.780,182.442420,8.899,2010.0,6.0,18.0


## Separate variable and target and preprocessings

In [63]:
target_name = "Weekly_Sales"

print("Separating labels from features...")
Y = df.loc[:, target_name]
X = df.drop(target_name, axis=1)
print("...Done.")
print(Y.head())
print()
print(X.head())
print()

Separating labels from features...
...Done.
0    1572117.54
1    1807545.43
4    1644470.66
6     695396.19
7    2203523.20
Name: Weekly_Sales, dtype: float64

   Store  Holiday_Flag  Temperature  Fuel_Price         CPI  Unemployment  \
0      6           NaN        59.61       3.045  214.777523         6.858   
1     13           0.0        42.38       3.435  128.616064         7.470   
4      6           0.0        78.89       2.759  212.412888         7.092   
6     15           0.0        69.80       4.069  134.855161         7.658   
7     20           0.0        39.93       3.617  213.023622         6.961   

     Year  Month   Day  
0  2011.0    2.0  18.0  
1  2011.0    3.0  25.0  
4  2010.0    5.0  28.0  
6  2011.0    6.0   3.0  
7  2012.0    2.0   3.0  



In [66]:
print("Dividing into train and test sets...")
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=0)
print("...Done.")
print()

Dividing into train and test sets...
...Done.



In [ ]:
numeric_features = ['Temperature', 'Fuel_Price', 'CPI', 'Unemployment', 'Year', 'Month', 'Day']
numeric_transformer = Pipeline(steps=[
    ("imputer",SimpleImputer(strategy="median")),
    ("scaler", StandardScaler()),
])

categorical_features = ['Store', 'Holiday_Flag']
categorical_transformer = Pipeline(steps=[
        ("imputer",SimpleImputer(strategy="most_frequent")),  # missing values will be replaced by most frequent value
        ("encoder",OneHotEncoder(drop="first"))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

In [69]:
print("Performing preprocessings on train set...")
print(X_train.head())
X_train = preprocessor.fit_transform(X_train)
print("...Done.")
print(X_train[0:5])
print()

print("Performing preprocessings on test set...")
print(X_test.head())
X_test = preprocessor.transform(X_test)
print("...Done.")
print(X_test[0:5, :])
print()

Performing preprocessings on train set...
     Store  Holiday_Flag  Temperature  Fuel_Price         CPI  Unemployment  \
127     16           0.0        61.79       2.711  189.523128         6.868   
63       5           0.0        69.17       3.594  224.019287         5.422   
35      19           0.0        33.26       3.789  133.958742         7.771   
10       8           0.0        82.92       3.554  219.070197         6.425   
95       1           0.0        74.78       2.854  210.337426         7.808   

       Year  Month   Day  
127  2010.0    7.0   9.0  
63   2012.0   10.0  19.0  
35   2011.0    3.0  25.0  
10   2011.0    8.0  19.0  
95   2010.0    5.0  14.0  
...Done.
[[ 0.04260362 -1.26840641  0.20507788 -0.55534542 -1.1763434   0.147002
  -0.86859506  0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.          1.          0.          0.          0.
   0.          0.        ]
 [ 0.4592769   

# Part 2 : Baseline model

## Train model Linear Regression

In [ ]:
lr = LinearRegression()
lr.fit(X_train, Y_train)
print("...Done.")

...Done.


In [ ]:
print("Predictions on training set...")
Y_train_pred = lr.predict(X_train)
print("...Done.")
print(Y_train_pred)
print()

print("Predictions on test set...")
Y_test_pred = lr.predict(X_test)
print("...Done.")
print(Y_test_pred)
print()

Predictions on training set...
...Done.
[ 611364.67099396  370577.26212486 1275740.37137492  879179.76718068
 1536772.70829879 1514868.79536837 1965323.8723865   602145.54012832
  948687.87405245 1089144.04045663 2125262.41163193  650336.60787243
 2145312.0623884   610712.16639662  517258.85415892  778674.43751482
  621000.71199925 1637887.71082182  166083.77933535  532890.97130511
 1846150.02967254 2113342.41663076 1117874.96097089 1449549.93545644
 2064847.33029364 1946434.88789985  420203.37186409 2018205.31305823
  911972.28740893 1619671.09250448 2039633.27775499 1566247.21290487
 1544871.47814238 1918280.17346583  329688.5413809   513754.34016273
  930146.16563808 1520404.73250487 2020147.89985993 2062163.07944381
  523043.15752523 1942173.83959015 1592843.57093178  425386.35441876
  245875.50172862  503128.68941671  438285.84900604 1792986.20265494
 1965095.38313836  420314.62383058 2068359.18918786 1881633.53078707
  798003.7668471  1545014.05935314  471641.00853216  408800.486

In [196]:
best_scores = pd.DataFrame(columns=['score', 'train', 'test'])

In [197]:
new = pd.DataFrame([['lr', r2_score(Y_train, Y_train_pred), r2_score(Y_test, Y_test_pred)]], columns=['score', 'train', 'test'])
best_scores = pd.concat([best_scores, new], ignore_index=True)

print("R2 score on training set : ", r2_score(Y_train, Y_train_pred))
print("R2 score on test set : ", r2_score(Y_test, Y_test_pred))

R2 score on training set :  0.9868321417045137
R2 score on test set :  0.9352216314000102


In [ ]:
col_names = list(preprocessor.transformers_[0][1].get_feature_names_out()) + list(preprocessor.transformers_[1][1].get_feature_names_out())

coefs = pd.DataFrame(
   lr.coef_,
   columns=['coefficients'], index = col_names
)

coefs = coefs.reset_index()

coefs.rename(columns={'index': 'name'}, inplace=True)

colors = ['Positive' if c > 0 else 'Negative' for c in lr.coef_]

fig = px.bar(y = coefs.coefficients, x = coefs.name, color = colors)

fig.show()

In [147]:
data = {
    'y_test' : Y_test,
    'y_test_pred_lr' : Y_test_pred
}

df_test = pd.DataFrame(data)


fig = px.scatter(df_test, x = 'y_test', y = 'y_test_pred_lr', trendline='ols')

fig.show()


## Part 3 : Fight overfitting

## Ridge

In [198]:
ridge = Ridge()
print(ridge)
ridge.fit(X_train, Y_train)

new = pd.DataFrame([['ridge', ridge.score(X_train, Y_train), ridge.score(X_test, Y_test)]], columns=['score', 'train', 'test'])
best_scores = pd.concat([best_scores, new], ignore_index=True)

print("R2 score on training set : ", ridge.score(X_train, Y_train))
print("R2 score on test set : ", ridge.score(X_test, Y_test))

Ridge()
R2 score on training set :  0.9326481680110413
R2 score on test set :  0.8246510243579797


## Lasso

In [199]:
lasso = Lasso()
print(lasso)
lasso.fit(X_train, Y_train)

new = pd.DataFrame([['lasso', lasso.score(X_train, Y_train), lasso.score(X_test, Y_test)]], columns=['score', 'train', 'test'])
best_scores = pd.concat([best_scores, new], ignore_index=True)

print("R2 score on training set : ", lasso.score(X_train, Y_train))
print("R2 score on test set : ", lasso.score(X_test, Y_test))

Lasso()
R2 score on training set :  0.9864577742465312
R2 score on test set :  0.939043642068452


## GridSearch on Ridge

In [201]:
regressor = Ridge()

params = {
    'alpha': [0.01, 0.02, 0.05, 0.1, 0.5, 1, 2, 3, 4, 5, 6, 7] 
}

gridsearch_R = GridSearchCV(regressor, param_grid = params, cv = 10, scoring = 'r2') 
gridsearch_R.fit(X_train, Y_train)
print("...Done.")
print("Best hyperparameters : ", gridsearch_R.best_params_)
print("Best R2 score : ", gridsearch_R.best_score_)

print("R2 score on training set : ", gridsearch_R.score(X_train, Y_train))
print("R2 score on test set : ", gridsearch_R.score(X_test, Y_test))

scores = cross_val_score(gridsearch_R.best_estimator_, X_train, Y_train, cv = 10)
print('The cross-validated R2-score is : ', scores.mean())
print('The standard deviation is : ', scores.std())

new = pd.DataFrame([['ridge_GS', gridsearch_R.score(X_train, Y_train), gridsearch_R.score(X_test, Y_test)]], columns=['score', 'train', 'test'])
best_scores = pd.concat([best_scores, new], ignore_index=True)

...Done.
Best hyperparameters :  {'alpha': 0.01}
Best R2 score :  0.9616884311634845
R2 score on training set :  0.9861304838628825
R2 score on test set :  0.9389731781408599
The cross-validated R2-score is :  0.9616884311634845
The standard deviation is :  0.022426850547857126


## GridSearch on Lasso

In [202]:
regressor = Lasso()

params = {'alpha': np.arange(0,100,1)}

gridsearch_L = GridSearchCV(regressor, param_grid = params, cv = 10, scoring = 'r2') 
gridsearch_L.fit(X_train, Y_train)
print("...Done.")
print("Best hyperparameters : ", gridsearch_L.best_params_)
print("Best R2 score : ", gridsearch_L.best_score_)

print("R2 score on training set : ", gridsearch_L.score(X_train, Y_train))
print("R2 score on test set : ", gridsearch_L.score(X_test, Y_test))

scores = cross_val_score(gridsearch_L.best_estimator_, X_train, Y_train, cv = 10)
print('The cross-validated R2-score is : ', scores.mean())
print('The standard deviation is : ', scores.std())

new = pd.DataFrame([['lasso_GS', gridsearch_L.score(X_train, Y_train), gridsearch_L.score(X_test, Y_test)]], columns=['score', 'train', 'test'])
best_scores = pd.concat([best_scores, new], ignore_index=True)

...Done.
Best hyperparameters :  {'alpha': np.int64(25)}
Best R2 score :  0.9614837366884833
R2 score on training set :  0.9863809358602692
R2 score on test set :  0.9392431171600182
The cross-validated R2-score is :  0.9614837366884833
The standard deviation is :  0.022613035919754287


In [203]:
best_scores

,score,train,test
0,lr,0.986832,0.935222
1,ridge,0.932648,0.824651
2,lasso,0.986458,0.939044
3,ridge_GS,0.986130,0.938973
4,lasso_GS,0.986381,0.939243
